# 🎯 Fine-Tuning

Adapter le modèle à des tâches spécifiques

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## Qu'est-ce que le Fine-Tuning ?

### Pré-entraînement vs Fine-tuning

**Pré-entraînement** :
- Large corpus (ex: tout Internet)
- Tâche générale : prédire le prochain token
- Apprentissage non supervisé
- Coûteux en temps et ressources

**Fine-tuning** :
- Dataset spécifique (ex: dialogues, code, médical)
- Tâche ciblée (ex: Q&A, classification, génération)
- Apprentissage supervisé
- Rapide et efficace

## Stratégies de Fine-Tuning

### 1. Full Fine-Tuning
Mettre à jour tous les poids du modèle.

In [ ]:
def full_fine_tuning(model, task_data, config):
    """
    Fine-tune tous les paramètres du modèle.
    
    Args:
        model: Modèle pré-entraîné
        task_data: Dataset spécifique à la tâche
        config: Configuration (learning rate, epochs, etc.)
    """
    optimizer = AdamOptimizer(learning_rate=config['lr'])
    
    for epoch in range(config['epochs']):
        for x, y in task_data:
            # Forward
            logits = model.forward(x)
            loss = cross_entropy_loss(logits, y)
            
            # Backward
            grads = model.backward(y)
            
            # Update ALL parameters
            optimizer.update(model.parameters, grads)
    
    return model

print("✅ Full Fine-Tuning: Met à jour tous les poids")
print("   Avantages: Meilleure adaptation")
print("   Inconvénients: Coûteux, risque de catastrophic forgetting")

### 2. Feature Extraction (Frozen Embeddings)
Geler les embeddings, entraîner seulement les couches supérieures.

In [ ]:
def feature_extraction_fine_tuning(model, task_data, config):
    """
    Fine-tune avec embeddings gelés.
    """
    optimizer = AdamOptimizer(learning_rate=config['lr'])
    
    # Freeze embeddings
    frozen_params = ['token_embedding', 'pos_encoding']
    
    for epoch in range(config['epochs']):
        for x, y in task_data:
            logits = model.forward(x)
            loss = cross_entropy_loss(logits, y)
            grads = model.backward(y)
            
            # Update only non-frozen parameters
            trainable_params = {k: v for k, v in model.parameters.items() 
                               if k not in frozen_params}
            trainable_grads = {k: v for k, v in grads.items() 
                              if k not in frozen_params}
            
            optimizer.update(trainable_params, trainable_grads)
    
    return model

print("❄️ Feature Extraction: Gèle les embeddings")
print("   Avantages: Plus rapide, moins de paramètres")
print("   Inconvénients: Adaptation limitée")

### 3. Layer-wise Learning Rate
Learning rates différents par couche.

In [ ]:
def layerwise_lr_fine_tuning(model, task_data, config):
    """
    Fine-tune avec learning rates différents par couche.
    
    Principe: Les couches basses (embeddings) changent peu,
              les couches hautes (output) changent plus.
    """
    # Learning rates décroissants par couche
    layer_lrs = {
        'token_embedding': config['lr'] * 0.1,
        'pos_encoding': config['lr'] * 0.1,
        'blocks': config['lr'] * 0.5,
        'output_proj': config['lr'] * 1.0
    }
    
    optimizers = {name: AdamOptimizer(learning_rate=lr) 
                 for name, lr in layer_lrs.items()}
    
    for epoch in range(config['epochs']):
        for x, y in task_data:
            logits = model.forward(x)
            loss = cross_entropy_loss(logits, y)
            grads = model.backward(y)
            
            # Update avec différents learning rates
            for layer_name, optimizer in optimizers.items():
                layer_params = {k: v for k, v in model.parameters.items() 
                               if layer_name in k}
                layer_grads = {k: v for k, v in grads.items() 
                              if layer_name in k}
                optimizer.update(layer_params, layer_grads)
    
    return model

print("📊 Layer-wise LR: Learning rates adaptatifs")
print("   Couches basses: LR faible (0.1x)")
print("   Couches intermédiaires: LR moyen (0.5x)")
print("   Couches hautes: LR fort (1.0x)")

## Exemple: Fine-Tuning pour Q&A

In [ ]:
# Dataset de Q&A
qa_dataset = [
    {"question": "What is the capital of France?", 
     "answer": "The capital of France is Paris."},
    {"question": "Who wrote Romeo and Juliet?", 
     "answer": "William Shakespeare wrote Romeo and Juliet."},
    {"question": "What is 2 + 2?", 
     "answer": "2 + 2 equals 4."},
]

# Format pour le fine-tuning
def prepare_qa_data(qa_dataset):
    """
    Formate les paires Q&A pour l'entraînement.
    
    Format: "Q: {question}\nA: {answer}"
    """
    formatted_data = []
    
    for item in qa_dataset:
        text = f"Q: {item['question']}\nA: {item['answer']}"
        
        # Encode
        tokens = encode(text)
        
        # Input = tout sauf dernier token
        # Target = tout sauf premier token
        x = tokens[:-1]
        y = tokens[1:]
        
        formatted_data.append((x, y))
    
    return formatted_data

# Préparer les données
train_data = prepare_qa_data(qa_dataset)

print(f"Dataset Q&A préparé: {len(train_data)} exemples")
print(f"\nExemple:")
print(f"  Question: {qa_dataset[0]['question']}")
print(f"  Réponse: {qa_dataset[0]['answer']}")

## Prévenir le Catastrophic Forgetting

### Techniques :

1. **Lower Learning Rate** : Utiliser un LR plus faible que le pré-entraînement
2. **Fewer Epochs** : Ne pas surentraîner
3. **Regularization** : L2, Dropout
4. **Mix General Data** : Mélanger données générales + spécifiques

In [ ]:
# Configuration pour éviter le catastrophic forgetting
safe_config = {
    'lr': 0.0001,  # 10x plus petit que pré-entraînement
    'epochs': 3,    # Peu d'epochs
    'batch_size': 16,
    'dropout': 0.1,
    'weight_decay': 0.01  # L2 regularization
}

print("🛡️ Configuration sûre pour le fine-tuning:")
for key, value in safe_config.items():
    print(f"  {key}: {value}")

## Évaluation du Fine-Tuning

In [ ]:
def evaluate_qa_model(model, test_questions):
    """
    Évalue le modèle fine-tuné sur des questions.
    """
    for question in test_questions:
        prompt = f"Q: {question}\nA:"
        
        # Générer la réponse
        answer = generate_text(model, prompt, max_length=20, 
                              strategy='top_p', p=0.9, temperature=0.7)
        
        print(f"Q: {question}")
        print(f"A: {answer}\n")

# Test questions
test_questions = [
    "What is the capital of Italy?",
    "Who painted the Mona Lisa?",
    "What is 10 + 5?"
]

print("🧪 Test du modèle fine-tuné:")
# evaluate_qa_model(finetuned_model, test_questions)

## Visualisation: Avant vs Après Fine-Tuning

In [ ]:
# Simuler les performances
metrics = {
    'General Tasks': [0.75, 0.73],  # Légère baisse acceptable
    'Q&A Task': [0.45, 0.92],       # Forte amélioration
}

fig, ax = plt.subplots(figsize=(10, 6))

x = np.arange(len(metrics))
width = 0.35

before = [v[0] for v in metrics.values()]
after = [v[1] for v in metrics.values()]

ax.bar(x - width/2, before, width, label='Avant Fine-Tuning', alpha=0.8)
ax.bar(x + width/2, after, width, label='Après Fine-Tuning', alpha=0.8)

ax.set_ylabel('Accuracy')
ax.set_title('Impact du Fine-Tuning')
ax.set_xticks(x)
ax.set_xticklabels(metrics.keys())
ax.legend()
ax.set_ylim([0, 1])

plt.tight_layout()
plt.show()

print("\n📈 Résultats:")
print("  • Tâches générales: Légère baisse (75% → 73%)")
print("  • Tâche Q&A: Forte amélioration (45% → 92%)")
print("  ✅ Trade-off acceptable !")